In [2]:
from sympy import Symbol, limit, oo
import numpy as np
import pandas as pd
%precision 6

'%.6f'

## Study Session 1 - Quantitative Methods 1

### Reading 1 - The Time Value of Money

Learning outcomes
e. "...(PV only)...": An infinite series of cash flows has no future value; for if they did, when?

#### Interest Rates
"In this section, we briefly explain the meaning and <u><i>interpretation</i></u>...": As a contextual meaning.

"2 Technically, 1 plus the nominal rate equals...": This identity in mathematical notation:
$$
1 + TheNominalRate = (1 + TheRealRate) \cdot (1 + TheInflationRate)
$$
and this is to be invistigated later on.

Findings: This equation is named __The Fischer Equation__ after the economist Fischer.
The rationale relates roughly to the purchasing power of a unit of currency. To be futher analysed. 

Translating equation 2 to a Python function for use in the following examples, noting that the equation is
$$
FV_N = PV(1 + r)^N
$$

In [3]:
def FV_N(PV, r, N):
    return PV * (1 + r)**N

##### Solution to Example 1


In [4]:
PV, r, N = 5e6, .07, 5

$FV_N = $

In [5]:
FV_N(PV, r, N)

7012758.653500

##### Solution to Example 2

In [6]:
PV, r, N = 2.5e6, 0.08, 6

$FV_N = $

In [7]:
FV_N(PV, r, N)

3967185.807360

##### Solution to Example 3

In [8]:
PV, r, N = 10e6, 0.09, 15 - 5

$FV_N = $

In [9]:
FV_N(PV, r, N)

23673636.745921

#### Non-Annual Compounding (Future Value)

Translating equation 2 to a Python function for use in the following examples, noting that the equation is
$$
FV_N = PV(1 + \frac{r_s}{m})^{mN}
$$

In [10]:
def FV_N_m(PV, r_s, m, N):
    return FV_N(PV, r_s/m, m*N)

##### Solution to Example 1

In [11]:
FV_N_m(10_000, 0.08, 4, 2)

11716.593810

##### Solution to Example 2

In [12]:
FV_N_m(1_000_000, 0.06, 12, 1)

1061677.811864

#### Continuous Compounding, Stated and Effictive Rates

Equation 4:
$$
FV_N = PV e^{r_s N}
$$

Defining the right-hand-side of equation 4 as a Python function:

In [13]:
def FV_N_continuous(PV, r_s, N):
   return PV * np.e**(r_s*N) 

##### Solution to Example 6

In [14]:
FV_N_continuous(10_000, 0.08, 2)

11735.108710

##### Stated and Effective Rates

For an insight into the difference between different compounding frequencies we replicate table 1 below
then we plot a graph to compare the effects of those frequencies assuming other variables fixed.

In [15]:
# Replicate Table 1
frequencies = [1, 2, 4, 12, 356, oo]
table_1 = pd.DataFrame(
    {
        "Frequency": ["Annual", "Semiannual", "Quarterly", "Monthly", "Daily", "Continuous"],
        "r_s/m": [0.08/f for f in frequencies],
        "mN": [f * 1 for f in frequencies]
    },
    
)
table_1

,Frequency,r_s/m,mN
0,Annual,0.08,1
1,Semiannual,0.04,2
2,Quarterly,0.02,4
3,Monthly,0.006667,12
4,Daily,0.000225,356
5,Continuous,0,oo


In [18]:
def FV_generic(PV, r_s, m, N):
    _m = Symbol('_m')
    return PV * limit((1 + r_s/_m)**(_m*N), _m, m).evalf()

table_1.apply(lambda row: FV_generic(1, 0.08, row['mN'], 1), axis=1)

0    1.08000000000000
1    1.08160000000000
2    1.08243216000000
3    1.08299950680751
4    1.08327733176541
5    1.08328706767496
dtype: object